In [ ]:

import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import spacy

 
def read_article(file_name):
    article = file_name.split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: 
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []
    sentences =  read_article(file_name)
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))
    summary=". ".join(summarize_text)
    return summary



In [ ]:
import spacy
def generate_keyword(summary):

  nlp = spacy.load("en_core_web_sm")
  doc = nlp(summary)
  return doc.ents[1].text

In [ ]:
!pip install flask-ngrok

In [ ]:
!sudo apt-get install python3-magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-mgc libmagic1 python3-magic
0 upgraded, 3 newly installed, 0 to remove and 12 not upgraded.
Need to get 262 kB of archives.
After this operation, 5,257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-magic all 2:0.4.15-1 [9,280 B]
Fetched 262 kB in 2s (157 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76

In [ ]:
pip install gTTS

In [ ]:
from gtts import gTTS 

def convert_summary_to_audio(summary_text, filename):
  mytext = summary_text
  language = 'en'
  myobj = gTTS(text=mytext, lang=language, slow=False) 
  myobj.save("static/"+filename+".mp3") 



In [ ]:
import os
import time
import urllib
import requests
import magic
import progressbar
from urllib.parse import quote

class simple_image_download:
    def __init__(self):
        pass

    def urls(self, keywords, limit, extensions={'.jpg', '.png', '.ico', '.gif', '.jpeg'}):
        keyword_to_search = [str(item).strip() for item in keywords.split(',')]
        i = 0
        links = []

        things = len(keyword_to_search) * limit

        # bar = progressbar.ProgressBar(maxval=things, \
                                      # widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()]).start()

        while i < len(keyword_to_search):
            url = 'https://www.google.com/search?q=' + quote(
                keyword_to_search[i].encode(
                    'utf-8')) + '&biw=1536&bih=674&tbm=isch&sxsrf=ACYBGNSXXpS6YmAKUiLKKBs6xWb4uUY5gA:1581168823770&source=lnms&sa=X&ved=0ahUKEwioj8jwiMLnAhW9AhAIHbXTBMMQ_AUI3QUoAQ'
            raw_html = self._download_page(url)

            end_object = -1;
            google_image_seen = False;
            j = 0

            while j < limit:
                while (True):
                    try:
                        new_line = raw_html.find('"https://', end_object + 1)
                        end_object = raw_html.find('"', new_line + 1)

                        buffor = raw_html.find('\\', new_line + 1, end_object)
                        if buffor != -1:
                            object_raw = (raw_html[new_line + 1:buffor])
                        else:
                            object_raw = (raw_html[new_line + 1:end_object])

                        if any(extension in object_raw for extension in extensions):
                            break

                    except Exception as e:
                        break


                try:
                    r = requests.get(object_raw, allow_redirects=True, timeout=1)
                    if('html' not in str(r.content)):
                        mime = magic.Magic(mime=True)
                        file_type = mime.from_buffer(r.content)
                        file_extension = f'.{file_type.split("/")[1]}'
                        if file_extension == '.png' and not google_image_seen:
                            google_image_seen = True
                            raise ValueError();
                        links.append(object_raw)
                        # bar.update(bar.currval + 1)
                    else:
                        j -= 1
                except Exception as e:
                    j -= 1
                j += 1

            i += 1

        # bar.finish()
        return(links)


    def download(self, keywords, limit, extensions={'.jpg', '.png', '.ico', '.gif', '.jpeg'}):
        keyword_to_search = [str(item).strip() for item in keywords.split(',')]
        main_directory = "static/"
        i = 0

        things = len(keyword_to_search) * limit

        # bar = progressbar.ProgressBar(maxval=things, \
                                      # widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

        # bar.start()

        while i < len(keyword_to_search):
            self._create_directories(main_directory, keyword_to_search[i])
            url = 'https://www.google.com/search?q=' + quote(
                keyword_to_search[i].encode('utf-8')) + '&biw=1536&bih=674&tbm=isch&sxsrf=ACYBGNSXXpS6YmAKUiLKKBs6xWb4uUY5gA:1581168823770&source=lnms&sa=X&ved=0ahUKEwioj8jwiMLnAhW9AhAIHbXTBMMQ_AUI3QUoAQ'
            raw_html = self._download_page(url)

            end_object = -1;
            google_image_seen = False;
            j = 0
            while j < limit:
                while (True):
                    try:
                        new_line = raw_html.find('"https://', end_object + 1)
                        end_object = raw_html.find('"', new_line + 1)

                        buffor = raw_html.find('\\', new_line + 1, end_object)
                        if buffor != -1:
                            object_raw = (raw_html[new_line+1:buffor])
                        else:
                            object_raw = (raw_html[new_line+1:end_object])

                        if any(extension in object_raw for extension in extensions):
                            break

                    except Exception as e:
                        break
                path = main_directory + keyword_to_search[i].replace(" ", "_")

                try:
                    r = requests.get(object_raw, allow_redirects=True, timeout=1)
                    if('html' not in str(r.content)):
                        mime = magic.Magic(mime=True)
                        file_type = mime.from_buffer(r.content)
                        file_extension = f'.{file_type.split("/")[1]}'
                        if file_extension not in extensions:
                            raise ValueError()
                        if file_extension == '.png' and not google_image_seen:
                            google_image_seen = True
                            raise ValueError()
                        file_name = str(keyword_to_search[i]) + "_" + str(j + 1) + file_extension
                        with open(os.path.join(path, file_name), 'wb') as file:
                            file.write(r.content)
                        # bar.update(bar.currval + 1)
                    else:
                        j -= 1
                except Exception as e:
                    j -= 1
                j += 1

            i += 1
        # bar.finish()


    def _create_directories(self, main_directory, name):
        name = name.replace(" ", "_")
        try:
            if not os.path.exists(main_directory):
                os.makedirs(main_directory)
                time.sleep(0.2)
                path = (name)
                sub_directory = os.path.join(main_directory, path)
                if not os.path.exists(sub_directory):
                    os.makedirs(sub_directory)
            else:
                path = (name)
                sub_directory = os.path.join(main_directory, path)
                if not os.path.exists(sub_directory):
                    os.makedirs(sub_directory)

        except OSError as e:
            if e.errno != 17:
                raise
            pass
        return

    def _download_page(self,url):

        try:
            headers = {}
            headers['User-Agent'] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
            req = urllib.request.Request(url, headers=headers)
            resp = urllib.request.urlopen(req)
            respData = str(resp.read())
            return respData

        except Exception as e:
            print(e)
            exit(0)

In [ ]:
%mkdir static -p
%mkdir templates -p

In [ ]:
from flask_ngrok import run_with_ngrok
import json
from flask import Flask, render_template, request, redirect, url_for
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

summaries=[['Democratic candidate Joe Biden Wednesday said he was optimistic of a win in the US presidential elections and thanked his supporters for their patience.  Both Trump and Biden have made expected gains in smaller states','../static/trump.jpeg','../static/trump.mp3'],
           ['However, while the victory ensured the Hyderabad team got a spot in the playoffs, it also knocked Kolkata Knight Riders out of the tournament. However, with a victory needed to qualify, Sunrisers Hyderabad skipper David Warner and Wriddhiman Saha ensured the team chased down a total of 150 at a canter without losing a single wicket','../static/srh.jpeg','../static/trump.mp3'],
           ['PayPal  this week laid out its vision for the future of its digital wallet platform and its PayPal and Venmo  apps.  What’s more, PayPal put timeline on the Honey integrations and the other updates it plans to roll out over the course of the next year','../static/paypal.jpeg','../static/paypal.mp3'],
           ['Scientists have developed a vaccine candidate for COVID-19 that produces "extremely high levels" of protective antibodies in animal models, an advance that may lead to a novel therapeutic to curb the pandemic.  According to the study, the molecular structure of the vaccine roughly mimics that of a virus, which may account for its enhanced ability to provoke an immune response','../static/covid.jpeg','../static/covid.mp3']]
images=['../static/trump.jpeg','../static/srh.jpeg','../static/paypal.jpeg','../static/covid.jpeg']
audios=['../static/trump.mp3','../static/srh.mp3','../static/paypal.mp3','../static/covid.mp3']

@app.route('/', methods=['POST', 'GET'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        print(password)
        print(email)
        return redirect('home')
    return render_template('login.html')


@app.route('/home', methods=['POST', 'GET'])
def index():
    if request.method == 'POST':
        # summary = json.dumps({"summary" : generate_summary(request.form['summary'], 2)})
        summary = generate_summary(request.form['summary'], 2)
        keyword=generate_keyword(summary)
        print(summary)
        print(keyword)
        generate_image = simple_image_download
        generate_image().download(keyword, 1,extensions={'.png'})
        # keyword = keyword.replace(" ", "_")
        image="../static/"+keyword.replace(" ", "_")+"/"+keyword+"_1.png"
        convert_summary_to_audio(summary,keyword.replace(" ", "_"))
        filename="../static/"+keyword.replace(" ", "_")+".mp3"
        return redirect(url_for('summary', summary=summary, image=image, audio=filename))
    return render_template('index.html')


@app.route('/summary', methods=['POST', 'GET'])
def summary():
    if request.method == 'POST':
        return redirect('posts')
    summary=request.args['summary']
    image=request.args['image']
    audio=request.args['audio']
    summaries.insert(0,[summary,image,audio])
    images.insert(0,image)
    audios.insert(0,audio)
    return render_template('summary.html', summary=summary, image=image, audio=audio)


@app.route('/posts', methods=['POST', 'GET'])
def posts():
    return render_template('posts.html',summaries=summaries,images=images)
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://aa4ad271de2f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Nov/2020 13:47:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:47:10] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:47:10] "GET /static/user.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:47:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2020 13:47:22] "POST / HTTP/1.1" 302 -


Pass@13856
admin@gmail.com


127.0.0.1 - - [16/Nov/2020 13:47:22] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:47:23] "GET /static/style1.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:47:42] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2020 13:47:42] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2020 13:47:43] "GET /ads.txt HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2020 13:47:59] "GET /home HTTP/1.1" 200 -


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Riding without a helmet can now lead to a 3-month suspension of your driving license in Karnataka
Yes, you got it right.Karnataka`s Transport Department has made helmets mandatory for all people riding on the two-wheelers and imposed a three-month suspension of driving license for non-compliance.As per new directives, helmets are mandatory for all two-wheeler riders who are over four years of age
State govt has imposed a 3-month suspension of driving license along with a penalty in case of violation,the State Department of Transport, Karnataka said in its order.The state transport department has also issued a directive to strictly enforce the old rules for the safety of the riders.As per the rule, Under the Central Motor Vehicle Act, motorcycle riders must wear a helmet
As per the Karnataka Motor Vehicle Rule, all bikers in the state wearing helmets are mandatory.This 

127.0.0.1 - - [16/Nov/2020 13:48:38] "POST /home HTTP/1.1" 302 -
127.0.0.1 - - [16/Nov/2020 13:48:39] "GET /summary?summary=Riding+without+a+helmet+can+now+lead+to+a+3-month+suspension+of+your+driving+license+in+Karnataka.+State+govt+has+imposed+a+3-month+suspension+of+driving+license+along+with+a+penalty+in+case+of+violation%2Cthe+State+Department+of+Transport%2C+Karnataka+said+in+its+order.The+state+transport+department+has+also+issued+a+directive+to+strictly+enforce+the+old+rules+for+the+safety+of+the+riders.As+per+the+rule%2C+Under+the+Central+Motor+Vehicle+Act%2C+motorcycle+riders+must+wear+a+helmet&image=..%2Fstatic%2Fthe_State_Department_of_Transport%2Fthe+State+Department+of+Transport_1.png&audio=..%2Fstatic%2Fthe_State_Department_of_Transport.mp3 HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:48:39] "GET /static/the_State_Department_of_Transport/the%20State%20Department%20of%20Transport_1.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2020 13:48:39] "GET /static/style3.css HTTP/1.1"